# mwb_flow.prep Package Example 1.1 Pull MetData with GRIDtool update
This notebook provides a complete example of acquiring meterology data and formatting input data required for running the monthly water balance flow (mwb_flow) model (Gray and McCabe, 2010).
This example is an update to Example 1 becasue the metdata.py code has been updated to include the GRIDtool function, grid_area_weighted_volume(). 

A subset of dates were used (1/1/2016 - 12/31/2016) to reduce the time spent getting gridmet data while testing.

First import necessary packages/modules. Note the metdata_update.py import which contains the updated get_gridmet_at_points() function. 

In [ ]:
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
# Note the metdata_update import here
from prep.metdata_update import get_gridmet_at_points
from prep.datafile import check_format

Initializing mwb_flow.prep module.


Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column gage station numbers was used since the numbers will later be used to merge meterologic data with streamflow data.

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
# This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.
exres = exres.to_crs(4326)

In [3]:
# This example provides a unique identifier column rather than using the default DataFrame index.
# TODO change the gdf_index_col to accept full gage ID numbers?
exres_met = get_gridmet_at_points(exres, 'gageID', start='2016-01-01', end='2016-12-31')

C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata_update.py:50: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))


Retrieving GridMET cells...


C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata_update.py:71: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid


70 unique GridMET cells found for 6 input features.
Fetching GridMET data for unique cells...


Cells: 100%|██████████| 70/70 [02:23<00:00,  2.05s/it]


In [12]:
exres_met

<xarray.Dataset> Size: 56kB
Dimensions:        (time: 366, location: 6)
Coordinates:
    lat            (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon            (location) float64 48B -114.4 -114.4 -114.2 ... -114.5 -114.3
    elev           (location) float64 48B 1.435e+03 1.229e+03 ... 1.741e+03
  * location       (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time           (time) datetime64[ns] 3kB 2016-01-01 ... 2016-12-31
    area           (location) float64 48B 34.26 186.8 57.96 102.1 185.3 139.0
Data variables:
    min_temp       (time, location) float64 18kB 255.8 255.4 ... 261.5 261.4
    max_temp       (time, location) float64 18kB 265.9 265.1 ... 269.4 269.4
    precip_volume  (time, location) float64 18kB 0.0 1.587e+07 ... 3.883e+08
Attributes:
    featureType:  timeSeries

A object with meterology data was made using the CreateInputFile class. This class initialization and methods have been edited from the pydlem.prep module which required lake parameters inputs. This class will be further updated to create a method for merg stream gage data. Note that the data frequency is daily and needs to be reduced to monthly. 
This cell is here to demonstrate/test if the updated CreateInputFile code is working for now.  

In [13]:
test = CreateInputFile(geoms=None, met_data=exres_met, met_source='from_file')
test.data

<xarray.Dataset> Size: 56kB
Dimensions:        (time: 366, location: 6)
Coordinates:
    lat            (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon            (location) float64 48B -114.4 -114.4 -114.2 ... -114.5 -114.3
    elev           (location) float64 48B 1.435e+03 1.229e+03 ... 1.741e+03
  * location       (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time           (time) datetime64[ns] 3kB 2016-01-01 ... 2016-12-31
    area           (location) float64 48B 34.26 186.8 57.96 102.1 185.3 139.0
Data variables:
    min_temp       (time, location) float64 18kB 255.8 255.4 ... 261.5 261.4
    max_temp       (time, location) float64 18kB 265.9 265.1 ... 269.4 269.4
    precip_volume  (time, location) float64 18kB 0.0 1.587e+07 ... 3.883e+08
Attributes:
    featureType:  timeSeries

The daily temperature and precipitation data are summarised to monthly frequencies below. The resample function allows for simple grouping and the application of mathamatical functions. However, doing so changes the dimentions of an array, which makes this tricky. Perhaps a diffrent workflow is needed. 

Here, the daily temperature is averaged to calculate the monthly temperature. 

In [14]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

<xarray.DataArray 'mo_temp' (time: 12, location: 6)> Size: 576B
array([[-2.67580645, -2.73321386, -2.65173697, -2.65211694, -3.10505865,
        -3.32720898],
       [ 1.02364532,  1.06590038,  1.50954907,  1.3231681 ,  0.35258621,
         0.07293853],
       [ 1.9218894 ,  2.05227001,  2.39950372,  2.13024194,  1.12258065,
         0.64424965],
       [ 7.77738095,  7.78234568,  7.99435897,  7.72541667,  6.77962121,
         6.24601449],
       [ 9.32327189,  9.54450418,  9.61761787,  9.30625   ,  8.4691349 ,
         7.76514727],
       [14.385     , 14.50462963, 15.00717949, 14.70458333, 13.29219697,
        13.01797101],
       [15.66036866, 15.76451613, 16.57022333, 16.21139113, 14.54032258,
        14.43134642],
       [15.93294931, 15.89396655, 16.55930521, 16.27308468, 14.73760997,
        14.70680224],
       [10.04880952, 10.27141975, 10.71064103, 10.40052083,  9.23348485,
         8.92398551],
       [ 4.61313364,  4.84832736,  5.22332506,  4.91864919,  4.00293255,
         3.57791024],
       [ 2.56619048,  2.39555556,  2.91897436,  2.77020833,  1.82030303,
         1.72326087],
       [-8.09930876, -7.96833931, -7.76315136, -7.9452621 , -8.22030792,
        -8.64915849]])
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.435e+03 1.229e+03 ... 1.402e+03 1.741e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
    area      (location) float64 48B 34.26 186.8 57.96 102.1 185.3 139.0
  * time      (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

And the daily precipitation summed to calcualte the montly precipitation. 

TODO The units should be double checked to make sure they are correct after being converted to a volume. 

In [15]:
# Calculate the monthly precipitation
monthly_precip = exres_met.precip_volume.resample(time = "MS").sum()

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Precipitation Volume', 'units': 'm^3'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

<xarray.DataArray 'mo_precip' (time: 12, location: 6)> Size: 576B
array([[6.32413645e+09, 1.83323137e+10, 8.61503505e+09, 1.55534657e+10,
        2.21130521e+10, 1.55141152e+10],
       [4.98259683e+09, 1.38552893e+10, 6.72961921e+09, 1.23562722e+10,
        1.75353235e+10, 1.16009186e+10],
       [7.82176232e+09, 2.09727310e+10, 1.05383315e+10, 1.91317494e+10,
        2.67281298e+10, 1.81333219e+10],
       [1.50593559e+09, 5.93934479e+09, 2.26222392e+09, 4.22709626e+09,
        6.71670709e+09, 4.64439015e+09],
       [5.11707484e+09, 2.04384445e+10, 8.03851516e+09, 1.50387250e+10,
        2.31624205e+10, 1.63881458e+10],
       [2.09458120e+09, 8.62908312e+09, 3.57547741e+09, 6.79019025e+09,
        1.05017429e+10, 7.19230658e+09],
       [1.16894519e+09, 5.05724298e+09, 1.83225118e+09, 3.41303893e+09,
        5.54683842e+09, 3.70770854e+09],
       [6.00752721e+08, 2.70786387e+09, 8.84459111e+08, 1.62755458e+09,
        2.66048191e+09, 2.04287490e+09],
       [2.91691754e+09, 1.08402141e+10, 4.66192068e+09, 8.52494726e+09,
        1.30239189e+10, 8.91473606e+09],
       [1.60327300e+10, 5.05626643e+10, 2.27740065e+10, 4.05028243e+10,
        6.03137381e+10, 4.24380176e+10],
       [6.28475931e+09, 1.76491127e+10, 8.47485322e+09, 1.53304738e+10,
        2.20693134e+10, 1.44105092e+10],
       [5.38561019e+09, 1.73716357e+10, 7.77609715e+09, 1.43518672e+10,
        2.11679156e+10, 1.44134890e+10]])
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.435e+03 1.229e+03 ... 1.402e+03 1.741e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
    area      (location) float64 48B 34.26 186.8 57.96 102.1 185.3 139.0
  * time      (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Attributes:
    standard_name:  Monthy Precipitation Volume
    units:          m^3

The monthly temperature and precipitation data are merged into a single dataset that can be used as an input for the mwb_flow model. This also includes latitude which is an input as well. 
We can also use the updated check_format function from pydlem and see that the coordinates and required data variables are accounted for. 

In [16]:
metdata_input = xr.merge([Monthly_Temp, Monthly_Precip])
check_format(metdata_input)

All necessary variables exist and are labeled properly.
All necessary coordinates exist and are labeled properly


Lastly, we can view the dataset.

In [17]:
metdata_input

<xarray.Dataset> Size: 1kB
Dimensions:    (location: 6, time: 12)
Coordinates:
    lat        (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon        (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev       (location) float64 48B 1.435e+03 1.229e+03 ... 1.741e+03
  * location   (location) int64 48B 2800 3000 9600 9500 1000 5500
    area       (location) float64 48B 34.26 186.8 57.96 102.1 185.3 139.0
  * time       (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
Data variables:
    mo_temp    (time, location) float64 576B -2.676 -2.733 ... -8.22 -8.649
    mo_precip  (time, location) float64 576B 6.324e+09 1.833e+10 ... 1.441e+10
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

A DataArray of the monthly discharge is imported from the mwb_flow.q_datafile modual. This data was summarized by month, labeled with stream gage station number identifiers, and concatinated.

Stream gage data is required for calibration but not as an input into the mwb_flow model so we can keep it seperate. Ideally, the code in mwb_flow.q_datafile will be updated to include code from the MTDNRdata package which is where discharge can be quarried and pulled from. Perhaps the current code can be a 'from file' option for bringing in calibration data.

In [18]:
Monthly_Q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
Monthly_Q

<xarray.DataArray 'mo_discharge' (time: 12, location: 6)> Size: 576B
array([[ 0.89162157,         nan,         nan,         nan,         nan,
         1.36746823],
       [ 1.64474905,         nan,         nan,         nan,         nan,
         2.66221454],
       [ 2.47149894,         nan,         nan,         nan,         nan,
         4.21204339],
       [ 5.03766337,         nan,         nan,         nan,         nan,
        10.46576099],
       [ 4.72735291,         nan,         nan,         nan,         nan,
        11.27744678],
       [ 2.08566097,  0.13213007,  2.37564526,  2.7439634 ,         nan,
         6.27293575],
       [ 1.07488637,  0.07577071,  1.38843688,  1.65897329,         nan,
         1.7664283 ],
       [ 0.5593601 ,  0.0408128 ,  0.6090988 ,  0.60482234,  0.98153449,
         0.12890484],
       [ 0.58416357,  0.03919744,  0.70052005,  0.42791986,  1.19420146,
         0.25669362],
       [ 1.15964166,  0.1106497 ,  1.45650749,  1.23831321,  2.49366941,
         2.21259999],
       [ 1.33875424,  0.11112472,  2.56705619,  2.03577972,  2.86007881,
         3.09934107],
       [ 1.43390593,         nan,         nan,         nan,  2.2987911 ,
                nan]])
Coordinates:
  * time      (time) datetime64[ns] 96B 2016-01-01 2016-02-01 ... 2016-12-01
  * location  (location) <U4 96B '1000' '2800' '3000' '5500' '9500' '9600'
Attributes:
    standard_name:  Monthly Discharge
    units:          m^3/s